# The Xception model architecture
![Xception Model](https://vitalab.github.io/article/images/xception/architecture.png)
1. Xception is a depthwise separable convolution network
2. Consists of 3 flows entry, middle and exit flows
3. There are shortcuts between the many separable Conv layers
4. Default **input shape is 299x299x3**

> Original paper introducing Xception
[Original Paper](https://arxiv.org/abs/1610.02357)

# Importing packages

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import shutil
import keras
import skimage.io
import keras.backend as K
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation,add
from tensorflow.keras.models import Model, Sequential
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomCrop, Rescaling, RandomTranslation
from keras import Sequential
from tqdm import tqdm
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

import tensorflow_addons as tfa

# Loading data

In [ ]:
root_dir = '../input/cassava-leaf-disease-classification'

train_df = pd.read_csv(os.path.join(root_dir, 'train.csv'))

train_df.head()

# Preprocessing data

In [ ]:
image_preprocessor = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomCrop(150,150),
    RandomTranslation(0.3, 0.3),
    RandomRotation(0.5),
    Rescaling(1./255)])

def custom_gen(batch_size, image_dir, h = 150, w = 150):
    
    start = 0
    end = batch_size
    images = train_df['image_id']
    labels = train_df['label']
    while 1:
        
        if end >= train_df.shape[0]:
            start = 0
            end = batch_size 
            continue
        else:
        
            batch = []

            if start == 0:
                names = images[:end]
                y = to_categorical(labels[:end], num_classes = 5)
            else:
                names = images[start:end]
                y = to_categorical(labels[start:end], num_classes = 5)

            for name in names:

                img = cv2.imread(os.path.join(image_dir,name))
                img = np.expand_dims(img, axis = 0)
                img = image_preprocessor(img)
                img = np.squeeze(img, axis = 0)
                batch.append(img)



            end = end + batch_size
            start = start +  batch_size


            yield np.array(batch), y

# Loading VGG19

In [ ]:
os.makedirs('/tmp/.keras/datasets')

In [ ]:
shutil.copytree("../input/keras-pretrained-models", "/tmp/.keras/models")

In [ ]:
weights_path = '../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
model_xception = Xception(weights = weights_path, input_shape=(150,150,3),include_top=False)

model_xception.trainable = False

# Model

In [ ]:
model=Sequential()

model.add(model_xception)
model.add(Dropout(0.3))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(tf.keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None))
model.add(Dropout(0.3))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(tf.keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None))
model.add(Dropout(0.3))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(tf.keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None))
model.add(Dense(5, activation = 'softmax'))

model.summary()

In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',verbose = 1,factor = 0.75, min_lr = 0.00001)

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics=['accuracy', tfa.metrics.F1Score(num_classes = 5)])

# Training model

In [ ]:
batch_size = 512
epochs = 20
steps_per_epoch = train_df.shape[0] // batch_size
train_img_dir = os.path.join(root_dir, 'train_images')
train_gen = custom_gen(batch_size, train_img_dir)
  
history = model.fit(train_gen, epochs = epochs, steps_per_epoch = steps_per_epoch,verbose = 1,callbacks=[lrd])

# Predictions and submission

In [ ]:
test_leaf = "../input/cassava-leaf-disease-classification/test_images"

test_names = pd.Series(os.listdir(test_leaf))


for j in range(3):

    for i in tqdm(range(len(test_names))):

        image = cv2.imread(os.path.join(test_leaf, test_names[i]))
        image = np.expand_dims(image, axis = 0)
        image = image_preprocessor(image)
        if i ==0:

            pred = model.predict(image)
        else:
            pred = np.concatenate([pred, model.predict(image)])
            
    if j ==0:
        final = pred
    else:
        final = final +pred
     
pred = pd.Series(np.argmax(final, axis = 1))


test_df = pd.concat([test_names, pred], axis = 1)
test_df = test_df.rename(columns = {0: 'image_id', 1: 'label'})

test_df.to_csv('submission.csv', index = False)